In [520]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_squared_error as mse

In [521]:
from sklearn.model_selection import train_test_split

In [659]:
data = pd.read_csv('input/train.csv')

In [660]:
data.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644


In [661]:
data = data.loc[data['Rooms']<10,:]

data = data.loc[data['Price'].between(30000,600000),:]

data['LifeSquare'] = data['LifeSquare'].fillna(data['Square']*data.loc[:,'LifeSquare'].mean()/data.loc[:,'Square'].mean())

data.loc[(data['Square']<15)&(data['LifeSquare']>15),'Square'] = data['LifeSquare']

data.loc[(data['Square']<15)&(data['LifeSquare']<15)&(data['Rooms']<=1),'Square'] = data.loc[data['Rooms']==1,'Square'].mean()

data.loc[(data['Square']<15)&(data['LifeSquare']<15)&(data['Rooms']==2),'Square'] = data.loc[data['Rooms']==2,'Square'].mean()

data.loc[(data['Square']<15)&(data['LifeSquare']<15)&(data['Rooms']==3),'Square'] = data.loc[data['Rooms']==3,'Square'].mean()

data.loc[(data['Square']<15)&(data['LifeSquare']<15)&(data['Rooms']==4),'Square'] = data.loc[data['Rooms']==4,'Square'].mean()

data.loc[(data['Square']<15)&(data['LifeSquare']<15)&(data['Rooms']==5),'Square'] = data.loc[data['Rooms']==5,'Square'].mean()

data.loc[(data['Square']>15)&(data['LifeSquare']<15),'LifeSquare'] = data['Square']

data.loc[data['Square']<data['LifeSquare'],'LifeSquare'] = data['Square']

data.loc[data['HouseYear']>2019,'HouseYear'] = data.loc[:,'HouseYear'].mean()

data.loc[data['KitchenSquare']>data['Square'],'KitchenSquare'] = data.loc[data['Rooms']==1,'KitchenSquare'].mean()

data.loc[data['Floor']>data['HouseFloor'],'HouseFloor'] = data['Floor']

data = pd.get_dummies(data)

data.columns

Index(['Id', 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare',
       'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Social_1', 'Social_2',
       'Social_3', 'Healthcare_1', 'Helthcare_2', 'Shops_1', 'Price',
       'Ecology_2_A', 'Ecology_2_B', 'Ecology_3_A', 'Ecology_3_B', 'Shops_2_A',
       'Shops_2_B'],
      dtype='object')

In [662]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9977 entries, 0 to 9999
Data columns (total 23 columns):
Id               9977 non-null int64
DistrictId       9977 non-null int64
Rooms            9977 non-null float64
Square           9977 non-null float64
LifeSquare       9977 non-null float64
KitchenSquare    9977 non-null float64
Floor            9977 non-null int64
HouseFloor       9977 non-null float64
HouseYear        9977 non-null float64
Ecology_1        9977 non-null float64
Social_1         9977 non-null int64
Social_2         9977 non-null int64
Social_3         9977 non-null int64
Healthcare_1     5185 non-null float64
Helthcare_2      9977 non-null int64
Shops_1          9977 non-null int64
Price            9977 non-null float64
Ecology_2_A      9977 non-null uint8
Ecology_2_B      9977 non-null uint8
Ecology_3_A      9977 non-null uint8
Ecology_3_B      9977 non-null uint8
Shops_2_A        9977 non-null uint8
Shops_2_B        9977 non-null uint8
dtypes: float64(9), int6

In [663]:
train, valid = train_test_split(data, test_size=0.3, random_state=42)

In [664]:
fts = ['Square','Rooms','LifeSquare','Ecology_1','Social_3','Helthcare_2','Shops_1','Ecology_2_A','Ecology_2_B','Ecology_3_A','Ecology_3_B','Shops_2_A','Shops_2_B','HouseYear','Floor','HouseFloor','KitchenSquare']

линейная регрессия

In [665]:
lr = LinearRegression()

In [666]:
lr.fit(train.loc[:,fts], train['Price'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [667]:
pred = lr.predict(train.loc[:,fts])

In [668]:
pred

array([273915.32799937, 298627.06710321, 256951.01092412, ...,
       213541.00386691, 166555.09530145, 119058.67110858])

In [669]:
r2(train['Price'], pred)

0.45861935422790767

In [670]:
pred_valid = lr.predict(valid.loc[:,fts])

In [671]:
r2(valid['Price'],pred_valid)

0.4635428019837441

RF

In [672]:
from sklearn.ensemble import RandomForestRegressor as RF

In [673]:
clf = RF(max_depth=18,n_estimators=40,max_features=3,random_state=100)

In [674]:
parameters = [{'n_estimators': [30,35,40,45,50,55,60],'max_features': np.arange(3,5),'max_depth': np.arange(10,30)}]

In [675]:
from sklearn.model_selection import GridSearchCV

In [676]:
clf = GridSearchCV(estimator=RF(random_state=100),param_grid=parameters,scoring='r2',cv=3)

In [677]:
clf.fit(train.loc[:,fts],train['Price'])

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=100, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'n_estimators': [30, 35, 40, 45, 50, 55, 60], 'max_features': array([3, 4]), 'max_depth': array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='r2', verbose=0)

In [678]:
clf.best_params_

{'max_depth': 17, 'max_features': 4, 'n_estimators': 60}

In [679]:
pred = clf.predict(train.loc[:,fts])

In [680]:
r2(train['Price'], pred)

0.9488631368647786

In [681]:
pred_valid = clf.predict(valid.loc[:,fts])

In [682]:
r2(valid['Price'],pred_valid)

0.7311893390424087

In [683]:
from sklearn.metrics import mean_squared_error as MSE

In [684]:
MSE(valid['Price'],pred_valid)

2130782701.0306773

дерево решений

In [548]:
from sklearn.tree import DecisionTreeRegressor

In [549]:
regressor = DecisionTreeRegressor(max_depth=21, random_state=100)

In [550]:
regressor.fit(train.loc[:,fts],train['Price'])

DecisionTreeRegressor(criterion='mse', max_depth=21, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=100, splitter='best')

In [551]:
pred = regressor.predict(train.loc[:,fts])

In [552]:
r2(train['Price'], pred)

0.995667452692434

In [553]:
pred_valid = regressor.predict(valid.loc[:,fts])

In [554]:
r2(valid['Price'],pred_valid)

0.47745261754293045

KNN

In [555]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor as KNN

масштабирование

In [556]:
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train.loc[:,fts])
valid_scaled = scaler.transform(valid.loc[:,fts])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


In [559]:
for i in range(2,22,2):
    print(f'n_neighbors={i}')
    knn = KNN(n_neighbors=i,weights='distance')
    knn.fit(train_scaled, train['Price'])
    pred = knn.predict(train_scaled)
    print(r2(train['Price'],pred))
    pred_valid = knn.predict(valid_scaled)
    print(r2(valid['Price'],pred_valid))
    print()

n_neighbors=2
1.0
0.5754657697325057

n_neighbors=4
1.0
0.625883031937986

n_neighbors=6
1.0
0.6283873834305418

n_neighbors=8
1.0
0.6283936631075152

n_neighbors=10
1.0
0.6313813758490938

n_neighbors=12
1.0
0.6285836709412811

n_neighbors=14
1.0
0.6241608946137631

n_neighbors=16
1.0
0.6220373480371482

n_neighbors=18
1.0
0.6178728933837597

n_neighbors=20
1.0
0.6181708382940052



Вывод: 
    самая лучшая модель Random Forest (случайный лес). 
    Она дает наилучшие предсказания и ее будем использовать для теста!

In [685]:
data = pd.read_csv('input/test.csv')

In [687]:
data.loc[data['Rooms']>9,'Rooms'] = 2

data['LifeSquare'] = data['LifeSquare'].fillna(data['Square']*data.loc[:,'LifeSquare'].mean()/data.loc[:,'Square'].mean())

data.loc[(data['Square']<15)&(data['LifeSquare']>15),'Square'] = data['LifeSquare']

data.loc[(data['Square']<15)&(data['LifeSquare']<15)&(data['Rooms']<=1),'Square'] = data.loc[data['Rooms']==1,'Square'].mean()

data.loc[(data['Square']<15)&(data['LifeSquare']<15)&(data['Rooms']==2),'Square'] = data.loc[data['Rooms']==2,'Square'].mean()

data.loc[(data['Square']<15)&(data['LifeSquare']<15)&(data['Rooms']==3),'Square'] = data.loc[data['Rooms']==3,'Square'].mean()

data.loc[(data['Square']<15)&(data['LifeSquare']<15)&(data['Rooms']==4),'Square'] = data.loc[data['Rooms']==4,'Square'].mean()

data.loc[(data['Square']<15)&(data['LifeSquare']<15)&(data['Rooms']==5),'Square'] = data.loc[data['Rooms']==5,'Square'].mean()

data.loc[(data['Square']>15)&(data['LifeSquare']<15),'LifeSquare'] = data['Square']

data.loc[data['Square']<data['LifeSquare'],'LifeSquare'] = data['Square']

data.loc[data['HouseYear']>2019,'HouseYear'] = data.loc[:,'HouseYear'].mean()

data.loc[data['KitchenSquare']>data['Square'],'KitchenSquare'] = data.loc[data['Rooms']==1,'KitchenSquare'].mean()

data.loc[data['Floor']>data['HouseFloor'],'HouseFloor'] = data['Floor']

data = pd.get_dummies(data)

data.columns

Index(['Id', 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare',
       'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Social_1', 'Social_2',
       'Social_3', 'Healthcare_1', 'Helthcare_2', 'Shops_1', 'Ecology_2_A',
       'Ecology_2_B', 'Ecology_3_A', 'Ecology_3_B', 'Shops_2_A', 'Shops_2_B'],
      dtype='object')

In [688]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 22 columns):
Id               5000 non-null int64
DistrictId       5000 non-null int64
Rooms            5000 non-null float64
Square           5000 non-null float64
LifeSquare       5000 non-null float64
KitchenSquare    5000 non-null float64
Floor            5000 non-null int64
HouseFloor       5000 non-null float64
HouseYear        5000 non-null float64
Ecology_1        5000 non-null float64
Social_1         5000 non-null int64
Social_2         5000 non-null int64
Social_3         5000 non-null int64
Healthcare_1     2623 non-null float64
Helthcare_2      5000 non-null int64
Shops_1          5000 non-null int64
Ecology_2_A      5000 non-null uint8
Ecology_2_B      5000 non-null uint8
Ecology_3_A      5000 non-null uint8
Ecology_3_B      5000 non-null uint8
Shops_2_A        5000 non-null uint8
Shops_2_B        5000 non-null uint8
dtypes: float64(8), int64(8), uint8(6)
memory usage: 654.4 KB


In [694]:
pred_test = clf.predict(data.loc[:,fts])

In [695]:
data['Price'] = pred_test

In [696]:
data.loc[:,['Id','Price']].to_csv('output/VZinchenko_predictions.csv',index=None)